In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
import panel as pn

pn.extension()


In [ ]:
# def callback(contents: str, user: str, instance: pn.chat.ChatInterface):
#     message = f"Echoing {user}: {contents}"
#     return message


# chat_interface = pn.chat.ChatInterface(
#     callback=callback, callback_user="System", show_clear=False
# )
# chat_interface.send("Send a message to receive an echo!", user="System", respond=False)
# chat_interface.servable()


In [ ]:
from llamabot import SimpleBot, ChatBot


bot = SimpleBot("You are Richard Feynman.")


async def callback(contents: str, user: str, instance: pn.chat.ChatInterface):
    response = bot(contents)
    yield response.content


chat_interface = pn.chat.ChatInterface(
    callback=callback, callback_user="Feynman Bot", show_clear=False
)
chat_interface.send(
    "Send a message to get a reply from the bot!",
    user="System",
    respond=False,
)
chat_interface.servable()


In [ ]:
from llamabot import QueryBot
import tempfile
import panel as pn
from pathlib import Path

pn.extension()

file_input = pn.widgets.FileInput(mime_type=["application/pdf"])
spinner = pn.indicators.LoadingSpinner(value=False, width=30, height=30)
global bot
bot = None


def upload_file(event):
    spinner.value = True
    raw_contents = event.new

    with tempfile.NamedTemporaryFile(
        delete=False, suffix=".pdf", mode="wb"
    ) as temp_file:
        temp_file.write(raw_contents)
        global bot
        bot = QueryBot("You are Richard Feynman", doc_paths=[Path(temp_file.name)])

    chat_interface.send(
        "Please allow me to summarize the paper for you. One moment...",
        user="System",
        respond=False,
    )
    response = bot("Please summarize this paper for me.")
    chat_interface.send(response.content, user="System", respond=False)
    spinner.value = False


file_input.param.watch(upload_file, "value")


async def callback(contents: str, user: str, instance: pn.chat.ChatInterface):
    spinner.value = True
    global bot
    response = bot(contents)
    spinner.value = False
    yield response.content


chat_interface = pn.chat.ChatInterface(
    callback=callback,
    callback_user="QueryBot",
    show_clear=False,
)
chat_interface.send(
    "Send a message to get a reply from the bot!",
    user="System",
    respond=False,
)

app = pn.Column(pn.Row(file_input, spinner), chat_interface)
app.show()
